In [6]:
import torch
import torchvision
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### data loader

In [2]:
transforms = []
transforms += [torchvision.transforms.Resize(224)]
transforms += [torchvision.transforms.RandomCrop(200)]
transforms += [torchvision.transforms.RandomHorizontalFlip()]
transforms += [torchvision.transforms.RandomVerticalFlip()]
transforms += [torchvision.transforms.RandomRotation(degrees=25)]
transforms += [torchvision.transforms.ToTensor()]
transforms += [torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])]
transforms = torchvision.transforms.Compose(transforms)

trainset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/valid/', transform=transforms)
validset = torchvision.datasets.ImageFolder(root='../../data/plant-seed/valid/', transform=None)

### resnet-34

In [3]:
model = torchvision.models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False    

model.fc = torch.nn.Linear(2048, 12)

### optimal learning rate

In [4]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)
learning_rates = [10**x/100000000 for x in range(0,10)]
criterion = torch.nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [5]:
losses = []

for learning_rate in learning_rates:
    # assign learning rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate
    # read data
    for i in range(6):
        _loss = 0.0 
        inputs, labels = next(iter(trainloader))
        if torch.cuda.is_available():
            inputs = Variable(inputs.cuda())
            labels = Variable(labels.cuda())
        else:
            inputs = Variable(inputs)
            labels = Variable(labels)

        # calculate loss and output
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _loss += loss.data[0]
    losses.append(_loss)
    
plt.figure(figsize=(10,5))
plt.plot(losses)
plt.xticks(range(len(losses)), learning_rates[:3])
plt.title('LR vs Loss')
plt.show()

### tune model

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)
learning_rates = [10**x/100000000 for x in range(0,10)]
criterion = torch.nn.CrossEntropyLoss()
optimizer= torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True, num_workers=4)
validloader = torch.utils.data.DataLoader(validset, batch_size=16, shuffle=True, num_workers=4)

In [ ]:
def score(model, dataloader):
    calc_loss = 0.
    calc_accuracy = 0.
    for data in dataloader: 
            inputs, labels = data
            if torch.cuda.is_available():
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs = Variable(inputs)
                labels = Variable(labels)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            calc_loss = loss.data[0]
            
            
def train(trainloader, validloader, epochs, interval):
    for epoch in range(10):
        # read data
        loss = 10
        for data in trainloader: 
            inputs, labels = data
            if torch.cuda.is_available():
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs = Variable(inputs)
                labels = Variable(labels)
            # calculate loss and output
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            # update weights
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if epoch % interval == 0:
            
        return model